In [ ]:
# Install dependencies
using Pkg
Pkg.add(["HDF5", "AWS", "AWSS3"]);

In [ ]:
using HDF5
using AWS
using AWSS3

In [ ]:
# Test a public file with https address via HDF5 module
# (h5open does NOT 
h5 = h5open("http://s3.eu-central-1.amazonaws.com/rhdf5-public/h5ex_t_array.h5"; driver=HDF5.Drivers.ROS3())
h5["DS1"][:]

In [ ]:
# This does NOT work, because Drivers.ROS3 only accepts a region with authentication (can't do public buckets this way)
h5 = h5open("s3://rhdf5-public/h5ex_t_array.h5"; driver=HDF5.Drivers.ROS3("eu-central-1", "", ""))

In [ ]:
# We could access our files like so:

In [ ]:
aws = global_aws_config() # get global AWS config

In [ ]:
# needs the http-type URL
url = "http://s3.$(aws.region).amazonaws.com/nasa-ghg-hub-scratch/absco/o2_v52.hdf"

In [ ]:
# Something like this should work maybe, but clearly doesnt.
# aws.credentials might not be what we want or need

h5 = h5open(url; driver=HDF5.Drivers.ROS3(aws.region, aws.credentials.access_key_id, aws.credentials.secret_key))